In [1]:
# 필요 모듈 임포트
import numpy as np
import pandas as pd
import gzip
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

%matplotlib inline

In [153]:
with gzip.open('../preprocessed/testSet.pickle','rb') as f:
    test = pickle.load(f)

In [155]:
test.columns = test.columns.drop('product_id').tolist()+['product_id']

In [ ]:
test.columns

In [ ]:
test_temp=pd.concat([test.iloc[:,:2],test.iloc[:,4:]],axis=1)
test_temp.columns


In [98]:
test=test_temp

In [99]:
test.shape

(3256262, 40)

In [ ]:
test.isna().sum()

In [101]:
with gzip.open('../preprocessed/final_data.pickle','rb') as f:
    final = pickle.load(f)

In [102]:
final['desired_amount_x'].equals(final['desired_amount_y'])

True

In [103]:
final = final.drop(['desired_amount_x'], axis=1)

In [ ]:
final_temp=pd.concat([final.iloc[:,0],final.iloc[:,3:]],axis=1)
final_temp.columns

In [109]:
final = final_temp

In [110]:
final.rename(columns={'desired_amount_y': 'desired_amount'}, inplace=True)

In [112]:
final.reset_index(inplace=True)
final.shape

(6833617, 40)

In [ ]:
# 열 순서 맞추기 
final_col=list(final.columns[:-1])
final_col.append('product_id')
final_col

In [ ]:
test=test[final_col]
test.head()

In [ ]:
# 결측치 학습데이터셋의 중앙값으로 대체하기
test.isna().sum()

In [ ]:
# test 데이터프레임 결측치 있는 열이름 저장
nan_cols=[]
for col in test.columns:
    if test[col].isna().sum()!=0:
        nan_cols.append(col)
nan_cols

In [ ]:
# test 데이터프레임 결측치 있는 열들은 final 데이터프레임의 같은 열 중간값으로 결측치 채우기
for col in nan_cols:
    test[col]=test[col].fillna(final[col].apply('median'))
test.isna().sum()

In [126]:
# 학습데이터셋에서 사용했던 OrdinalEncoder 동일하게 적용
with open('../processed/ordinal_encoder.pickle','rb') as f:
    ordinal_encoder = pickle.load(f)

In [ ]:
# 연속형/범주형 변수 뽑기
cat_columns = [c for c, t in zip(test.dtypes.index, test.dtypes) if t == 'O']
num_columns = [c for c    in test.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
# print('Numeric Columns: \n{}'.format(num_columns))

In [ ]:
# OrdinalEncoder 변환 순서에 맞게 test 데이터프레임도 변환
# purpose idx: 3, income_type idx: 0, employment_type idx: 1
ordinal_encoder.categories_

In [ ]:
oe_categories=ordinal_encoder.categories_
oe_categories[3]

In [ ]:
# purpose 인코딩 (purpose idx: 3)
for i in range(len(oe_categories[3])):
    test['purpose']=test['purpose'].replace(oe_categories[3][i],i) 
test['purpose'].head()

In [ ]:
# income_type 인코딩 (income_type idx: 0)
for i in range(len(oe_categories[0])):
    test['income_type']=test['income_type'].replace(oe_categories[0][i],i) 
test['income_type'].head()

In [ ]:
# employment_type 인코딩 (employment_type idx: 1)
for i in range(len(oe_categories[1])):
    test['employment_type']=test['employment_type'].replace(oe_categories[1][i],i) 
test['employment_type'].head()

In [133]:
# 최종 평가데이터셋에 필요한 feature만 남기기
X = test.drop(['application_id', 'user_id', 'product_id'], axis = 1)
# 제출할 테이블과 맞추기 위해 템플릿 따로 빼놓기
submit_template = test[['application_id','product_id']]
X.shape, submit_template.shape

((3256262, 37), (3256262, 2))

In [134]:
# 학습데이터에서 fit한 scaler 이용해서 평가데이터에 transform
with open('../preprocessed/scaler_final.pickle','rb') as f:
    scaler = pickle.load(f)

In [135]:
X_test_scaled = scaler.transform(X)

In [136]:
# 모델 불러오기
with open('../preprocessed/lgbm.pickle','rb') as f:
    lgbm = pickle.load(f)
    
with open('../preprocessed/random_forest.pickle','rb') as f:
    rf = pickle.load(f)

with open('../preprocessed/xgb.pickle','rb') as f:
    xgb = pickle.load(f)

In [137]:
# predict_proba() 통해 세 모델의 확률값 평균
y_proba_lgbm=lgbm.predict_proba(X_test_scaled)
y_proba_rf=rf.predict_proba(X_test_scaled)
y_proba_xgb=xgb.predict_proba(X_test_scaled)

In [144]:
all_test_proba=y_proba_lgbm[:,1]
all_test_proba+=y_proba_rf[:,1]
all_test_proba+=y_proba_xgb[:,1]
all_test_proba = all_test_proba/3
len(all_test_proba)

3256262

In [145]:
# threshold 조절하여 0 또는 1로 분류
@np.vectorize
def is_positive(x, thres):
    if x>thres:
        y = 1
    else:
        y = 0
    return y

In [146]:
y_pred = is_positive(all_test_proba, 0.48)
len(y_pred)

3256262

In [149]:
# 예측값의 불균형 여부 확인하기
import collections
print(collections.Counter(y_pred))

Counter({0: 2806059, 1: 450203})


In [ ]:
# 예측치를 하나의 데이터프레임으로 완성
submit_template['is_applied']=y_pred
submit_template

In [ ]:
# submit기준으로 조인 , 사이즈 확인 만약 사이즈 변했으면 drop duplicate
submission = pd.read_csv('../preprocessed/submission.csv')
print(submission.shape)
display(submission.head())

In [171]:
# submit 기준 조인
result=pd.merge(submission, submit_template, how='left', on=['application_id','product_id'])

In [172]:
result.shape

(3255254, 4)

In [ ]:
result.drop('is_applied_x',axis=1, inplace=True)
result.rename(columns={'is_applied_y':'is_applied'}, inplace=True)
result.head()

In [174]:
result=result.drop_duplicates()

In [175]:
result=result.reset_index(drop=False)
result=result.sort_values(by=['application_id','product_id','is_applied'])
result=result.drop_duplicates(subset=['application_id','product_id'], keep='first')
result=result.set_index('index').sort_index()
result.shape


(3255194, 3)

In [ ]:
result.head()

In [177]:
result['is_applied'].value_counts()

0    2805772
1     449422
Name: is_applied, dtype: int64

In [178]:
result.to_csv('../preprocessed/final_submission.csv', index=False)